In [ ]:
%load_ext autoreload

In [40]:
from linear_models import LinearOneVsAllClassifier
import numpy as np
import noise_functions_multi as noise
import time
import ray
from utils import find_noise_bounds_multi
%autoreload 2

In [ ]:
ray.init()

In [4]:
linear_models = [LinearOneVsAllClassifier(10, np.load('linear_models/multi/w_{}.npy'.format(i)), np.load('linear_models/multi/b_{}.npy'.format(i)))
                 for i in range(5)]

In [5]:
X_exp = np.load('linear_experiments_data/multi/X_exp.npy')
Y_exp = np.load('linear_experiments_data/multi/Y_exp.npy')

In [ ]:
linear_models[0].predict(X_exp[:1])

In [74]:
v = noise.grad_desc_nonconvex([1], linear_models[:1], X_exp[0], Y_exp[0], .2)

In [75]:
print v.shape
print np.linalg.norm(v)

(784,)
0.20901015931470487


In [76]:
linear_models[0].predict((X_exp[0] + v).reshape(1,-1))

array([7])

In [ ]:
mnist_images

In [42]:
find_noise_bounds_multi(linear_models[:1], X_exp[:1])

array([0.41317499])

In [8]:
np.linalg.norm(v[1])

3.4011209432506035

In [ ]:
s = time.time()
results = [noise.grad_desc_nonconvex.remote([1,1,1,1,1], linear_models, X_exp[i], Y_exp[i], 1) for i in xrange(500)]
res = ray.get(results)
print time.time() - s 

In [ ]:
s = time.time()
results = [noise.grad_desc_nonconvex([1,1,1,1,1], linear_models, X_exp[i], Y_exp[i], 1) for i in xrange(500)]
print time.time() - s 

In [ ]:
[model.evaluate((X_exp[0] + v).reshape(1,-1), Y_exp[0]) for model in linear_models]